# DX 704 Week 10 Project

In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [1]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

fatal: destination path 'SQuAD-explorer' already exists and is not an empty directory.


In [2]:
import json

In [3]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp, )

In [4]:
type(train_data)

dict

In [5]:
list(train_data.keys())

['data', 'version']

In [6]:
type(train_data["data"])

list

In [7]:
len(train_data["data"])

442

In [8]:
type(train_data["data"][0])

dict

In [9]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [10]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [11]:
len(train_data["data"][0]["paragraphs"])

55

In [12]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [13]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [14]:
with open('pretty_json.json', 'w') as f:
    json.dump(train_data, f, indent=4)

In [15]:
# Parse the contents of train_data to create a new file called "parsed.tsv"
import pandas as pd

parsed_file = []

# loop through josn to find all titles
for doc in train_data['data']:
    title = doc['title']
    for idx, paragraph in enumerate(doc['paragraphs']):
        context = paragraph['context']
        parsed_file.append({
            'document_title': title,
            'paragraph_index': idx,
            'paragraph_context': context
        })

# convert to dataframe
df = pd.DataFrame(parsed_file)
df.to_csv('submission/parsed.tsv', sep='\t', index=False)

# print preview of the data
print(f"Created parsed.tsv with {len(df)} rows.")
print("Sample of the first couple rows:")
print(df.head())

Created parsed.tsv with 18896 rows.
Sample of the first couple rows:
             document_title  paragraph_index  \
0  University_of_Notre_Dame                0   
1  University_of_Notre_Dame                1   
2  University_of_Notre_Dame                2   
3  University_of_Notre_Dame                3   
4  University_of_Notre_Dame                4   

                                   paragraph_context  
0  Architecturally, the school has a Catholic cha...  
1  As at most other universities, Notre Dame's st...  
2  The university is the major seat of the Congre...  
3  The College of Engineering was established in ...  
4  All of Notre Dame's undergraduate students are...  


Submit "parsed.tsv" in Gradescope.

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [16]:
# Design paragraph vectors based on the text length 1024
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1024)
X = vectorizer.fit_transform(df['paragraph_context'])

Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [17]:
# Save the vectors in a new file
df_X = pd.DataFrame(X.toarray())
df_vectors = pd.DataFrame({'document_title': df['document_title'], 'paragraph_index': df['paragraph_index'], 'paragraph_vector_json': df_X.values.tolist()})
df_vectors.to_csv('submission/paragraph-vectors.tsv.gz', sep='\t', index=False)

Submit "paragraph-vectors.tsv.gz" in Gradescope.

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [18]:
# Read the questions file
df_questions = pd.read_csv('questions.tsv', sep='\t')
questions_json = vectorizer.fit_transform(df_questions['question'])
df_questions.head()

,question_id,question
0,1,What was the goal of the abuse of region project?
1,4,How many satellites in the Beidou-1 constellat...
2,7,When did Beyoncé receive ten nominations for ...
3,10,"With which goddess did Sulla, Pompey, and Juli..."
4,13,What area is considered to have a desert clima...


Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

In [19]:
# Save the question vectors in a new file
df_ques_vec = pd.DataFrame({'question_id': df_questions['question_id'], 'question_vector_json': questions_json.toarray().tolist()})
df_ques_vec.to_csv('submission/question-vectors.tsv', sep='\t', index=False)

Submit "question-vectors.tsv" in Gradescope.

## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [20]:
# YOUR CHANGES HERE

...

Ellipsis

Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


In [21]:
# YOUR CHANGES HERE

...

Ellipsis

Submit "question-matches.tsv" in Gradescope.

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


Submit "worst-paragraphs.tsv" in Gradescope.

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.